# Clustering using the k-Means Clustering Algorithm

| Key              | Value                                                                                                                                                                                                                                                                                                        |
|:-----------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Course Codes** | BBT 4206 and BFS 4102                                                                                                                                                                                                                                                                                        |
| **Course Names** | BBT 4206: Business Intelligence II (Week 1-3 of 13) and<br/>BFS 4102: Advanced Business Data Analytics (Week 4-6 of 13)                                                                                                                                                                                      |
| **Semester**     | August to November 2025                                                                                                                                                                                                                                                                                      |
| **Lecturer**     | Allan Omondi                                                                                                                                                                                                                                                                                                 |
| **Contact**      | aomondi@strathmore.edu                                                                                                                                                                                                                                                                                       |
| **Note**         | The lecture contains both theory and practice.<br/>This notebook forms part of the practice.<br/>It is intended for educational purposes only.<br/>Recommended citation: [BibTex](https://raw.githubusercontent.com/course-files/ClusteringandAssociationRuleMining/refs/heads/main/RecommendedCitation.bib) |

**Business context**: A business aims to refine its marketing strategy by grouping customers into distinct segments based on _demographic details_ and _spending behavior_. This segmentation strategy serves as a foundation for implementing highly personalized marketing campaigns, where the leading KPI is the _customer engagement with tailored adverts_, and the lagging KPI is the _increase in overall customer spending_. By identifying specific customer clusters, the marketing team can optimize advertising efforts to target each group more effectively, ultimately driving increased customer satisfaction and revenue.

**Dataset**: The dataset used is based on the **"Mall Customers"** dataset, which consists of 200 observations. Each observation represents a customer, and the dataset includes demographic and behavioral features that are essential for customer segmentation.

| **Feature Type** | **Feature Name**         | **Description**                                                                              |
|:-----------------|:-------------------------|:---------------------------------------------------------------------------------------------|
| **Feature**      | `CustomerID`             | A unique identifier assigned to each customer                                                |
| **Feature**      | `Genre`                  | Gender of the customer (Male or Female)                                                      |
| **Feature**      | `Age`                    | Age of the customer in years                                                                 |
| **Feature**      | `Annual Income (k$)`     | Annual income of the customer in thousands of dollars                                        |
| **Feature**      | `Spending Score (1-100)` | Spending behavior of the customer on a scale of 1 to 100, with 100 being the highest spender |

Suppose that the dataset has been provided by the organization's customer service department for clustering purposes. This is done to segment customers into distinct groups based on their demographics (`Age`, `Genre`) and spending behaviors (`Annual Income`, `Spending Score`).

`import pandas as pd`
- 'Pandas' is essential because it is used for:
    - Loading the CSV data (`pd.read_csv()`)
    - Data manipulation and analysis
    - Creating and managing DataFrames
    - Essential for structured data analysis, e.g., `Group By` operations

`import matplotlib.pyplot as plt`
- 'Matplotlib' is used for:
    - Visualising data
    - Plotting graphs (e.g., scatter plots, line plots, the elbow method graph)
    - Customising visualisations (labels, titles, legends)

`from sklearn.preprocessing import StandardScaler, MinMaxScaler`
- 'StandardScaler' is used for:
    - Scaling features to have a mean of 0 and standard deviation of 1
    - This **data transformation** (referred to as standardization) is necessary to ensure that all features contribute equally to the distance calculations when clustering
- 'MinMaxScaler' is used for:
    - Normalization to ensure numeric features range between 0 and 1
- One-Hot encoding is another data transformation technique that converts categorical variables into binary vectors. It is not applied in this case

`from sklearn.cluster import KMeans`
- 'KMeans' is used for:
    - Performing K-Means clustering
    - Finding clusters in the data based on the features provided
    - The `fit_predict()` method can then be used to fit the model and predict cluster labels for each data point

`import seaborn as sns`
- 'Seaborn' is used for:
    - Enhanced data visualisation
    - Creating more aesthetically pleasing and informative plots (e.g., histograms, box plots, heatmaps)
    - It provides a high-level interface for drawing attractive statistical graphics

`import warnings`
`warnings.filterwarnings('ignore')`
- 'Warnings' is used to suppress warnings that may arise during the execution of the code, such as convergence warnings from K-Means. Even though it is not necessary for the clustering, it helps keep the output clean and focused on the results.

## Step 1: Import the necessary libraries

In [ ]:
# For file and system operations
import urllib.request
import os

# For data manipulation
import pandas as pd
import numpy as np

# For statistical data analysis
from scipy.stats import kurtosis
from scipy.stats import skew

# For data preprocessing and transformation
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

# For Machine Learning
from sklearn.cluster import KMeans

# For data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# For model persistence
import joblib

# For suppressing warnings
import warnings
warnings.filterwarnings('ignore')

## Step 2: Load the data

In [ ]:
dataset_path = './data/mall_customers.csv'
url = 'https://raw.githubusercontent.com/course-files/ClusteringandAssociationRuleMining/refs/heads/main/data/mall_customers.csv'

if not os.path.exists(dataset_path):
    print("Downloading dataset...")
    if not os.path.exists('./data'):
        os.makedirs('./data')
    urllib.request.urlretrieve(url, dataset_path)
    print("✅ Dataset downloaded")
else:
    print("✅ Dataset already exists locally")

use_cols = ['CustomerID', 'Genre', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']
customer_data = pd.read_csv(dataset_path, usecols=use_cols, encoding='utf-8', nrows=200000)

### Identify the numeric and categorical columns

**Selection of numeric columns**
- The code identifies columns with numeric data types (`int64` and `float64`) that can be subjected to mathematical or statistical functions.
- The code also identifies non-numeric columns (e.g., `strings`, `objects`, etc.) by excluding numeric (`int64`, `float64`) and `datetime` data types.
- This is done using `select_dtypes()` method of the DataFrame, which filters columns based on their data types.

In [ ]:
numeric_cols = customer_data.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = customer_data.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]']).columns

print("\nThe identified numeric columns are:")
print(numeric_cols.tolist())

print("\nThe identified categorical columns is/are:")
print(categorical_cols.tolist())

## Step 3: Initial Exploratory Data Analysis (EDA)

In [ ]:
print("\n*1* The number of observations and variables")
display(customer_data.shape)

print("\n*2* The data types:")
display(customer_data.info())

print("\n*3* The summary of the numeric columns:")
display(customer_data.describe())

print("\n*4* The whole dataset:")
display(customer_data)

print("\n*5* The first 5 rows in the dataset:")
display(customer_data.head())

print("\n*6* Percentage distribution for each category")
print("\nNumber of observations per class:")
print("Frequency counts:\n", customer_data['Genre'].value_counts())
print("\nPercentages:\n", customer_data['Genre'].value_counts(normalize=True) * 100, "%")

### Measures of Distribution

#### Variance of numeric columns

In [ ]:
print("\nVariance of numeric columns:")
print(customer_data[numeric_cols].var())

#### Standard deviation of numeric columns

In [ ]:
print("\nStandard deviation of numeric columns:")
print(customer_data[numeric_cols].std())

#### Kurtosis of numeric columns

In [ ]:
print("\nFisher Kurtosis of numeric columns:")
print("\nInterpretation:")
print("→ Positive kurtosis indicates heavier tails (more outliers) than what is expected in a normal distribution - leptokurtic")
print("→ Negative kurtosis indicates lighter tails (less outliers) than what is expected in a normal distribution - platykurtic")
print("→ A normal distribution has kurtosis of 0 - mesokurtic")
print("\nKurtosis values:")
print(customer_data[numeric_cols].apply(lambda x: kurtosis(x, fisher=True)))

#### Skewness of numeric columns

In [ ]:
print("\nSkewness of numeric columns:")
print("\nInterpretation:")
print("→ Positive skewness indicates a long right tail (right-skewed distribution)")
print("→ Negative skewness indicates a long left tail (left-skewed distribution)")
print("→ Skewness close to 0 indicates a symmetric distribution")
print("\nSkewness values:")
print(customer_data[numeric_cols].apply(lambda x: skew(x)))

### Measures of Relationship

#### Covariance matrix of numeric features

In [ ]:
print("\nCovariance matrix of numeric features:")
print("\nInterpretation:")
print("→ Positive values indicate that variables move in the same direction")
print("→ Negative values indicate that variables move in opposite directions")
print("→ Values close to 0 indicate little to no linear relationship")
print("\nCovariance values:")
display(customer_data[numeric_cols].cov())

#### Correlation matrix of numeric features

In [ ]:
print("\nSpearman correlation matrix of numeric features:")
spearman_corr = customer_data[numeric_cols].corr(method='spearman')
print("\nInterpretation:")
print("→ Values range from -1 to +1")
print("→ +1 indicates perfect positive correlation")
print("→ -1 indicates perfect negative correlation")
print("→ 0 indicates no correlation")
print("\nCorrelation values:")
display(spearman_corr)

### Basic visualization of the data

#### Histograms

In [ ]:
plt.figure(figsize=(15, 5))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(1, len(numeric_cols), i)
    sns.histplot(data=customer_data, x=col)
    plt.title(f'Distribution of {col}')
plt.tight_layout()
plt.show()

#### Box plots

In [ ]:
plt.figure(figsize=(12, 4))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(1, len(numeric_cols), i)
    sns.boxplot(data=customer_data, y=col)
    plt.title(f'Box Plot of {col}')
plt.tight_layout()
plt.show()


#### Missing data plot

In [ ]:
plt.figure(figsize=(8, 4))
sns.heatmap(customer_data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.title('Missing Data')
plt.show()

#### Correlation heatmap

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

#### Scatter plot matrix

In [ ]:
plt.figure(figsize=(10, 10))
sns.pairplot(customer_data[['CustomerID', 'Age', 'Annual Income (k$)', 'Spending Score (1-100)']])
plt.suptitle('Scatter Plot Matrix', y=1.02)
plt.show()

## Step 4: Feature selection for clustering
We will use **Age**, **Annual Income (KES)**, and **Spending Score (1‑100)**.

In [ ]:
X = customer_data[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']].copy()
X.head()

## Step 5: Data transformation

In [ ]:
# Option 1: Standardize (mean=0, variance=1)
scaler_standard = StandardScaler()
X_scaled = scaler_standard.fit_transform(X)
print("\nStandardized data (mean=0, variance=1):")

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
print("\nFirst 5 rows:")
display(X_scaled_df.head())
print("\nMean and standard deviation:")
display(X_scaled_df.describe())


# Option 2: Normalize (scale between 0 and 1) - this is optional for k-Means
# normalizer = MinMaxScaler()
# X_normalized = normalizer.fit_transform(X)
# print("\nNormalized data (scale between 0 and 1):")
# print(X_normalized)

## Step 6: Determine the optimal number of clusters using the Elbow method

1. **Inertia Calculation Loop:**
   - Creates an empty list `inertia` to store the sum of squared distances
   - Loops through k values from 1 to 10 using `range(1,11)`
   - For each k value:
     - Creates a KMeans model with k clusters
     - Fits the model to the scaled data
     - Adds the inertia (Within-Cluster Sum of Squares [WCSS]) to the list
          - **Within-Cluster Sum of Squares (WCSS)**, also known as **inertia**, measures the compactness of clusters in a clustering algorithm. It represents the sum of the squared distances between each data point in a cluster and the centroid of that cluster. A lower WCSS indicates that the data points within each cluster are closer to their respective centroid, suggesting tighter, more compact clusters.

2. **Visualization:**
   - Creates a figure of size 6x4
   - Plots k values (x-axis) against inertia values (y-axis)
   - Adds markers 'o' at each point
   - Labels axes and adds title
   - Displays the plot

3. **Purpose:**
   - The Elbow Method helps in finding the optimal number of clusters
   - The "elbow" in the plot indicates where adding more clusters gives diminishing returns
   - Point where the line starts to level out suggests optimal k value

In [ ]:
inertia = []
K = range(1, 11)
for k in K:
    km = KMeans(n_clusters=k, random_state=53)
    km.fit(X_scaled)
    inertia.append(km.inertia_)

plt.figure(figsize=(6,4))
plt.plot(K, inertia, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.title('Elbow Method for Finding the \"Optimal K\" in K-Means Clustering')
plt.show()

Based on the elbow plot, **k = 5** gives the ideal number of clusters; where adding more clusters does not significantly improve tightness.

## Step 7: Apply K-Means clustering using the optimal number of clusters

In [ ]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=53)
clusters = kmeans.fit_predict(X_scaled)
customer_data['Cluster'] = clusters
customer_data.head()
customer_data.to_csv('./data/mall_customers_with_clusters.csv')

## Step 8: Visualize the clusters

In [ ]:
plt.figure(figsize=(6,4))
scatter = plt.scatter(customer_data['Annual Income (k$)'],
                      customer_data['Spending Score (1-100)'],
                      c=customer_data['Cluster'], s=25)
plt.xlabel('Annual Income')
plt.ylabel('Spending Score (1‑100)')
plt.title('Customer Segments by Spending Score and Annual Income')
plt.legend(*scatter.legend_elements(), title='Cluster', loc='upper right',
           bbox_to_anchor=(1.15, 1), fontsize='small')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
scatter = plt.scatter(customer_data['Age'],
                      customer_data['Spending Score (1-100)'],
                      c=customer_data['Cluster'], s=25)
plt.xlabel('Age')
plt.ylabel('Spending Score (1‑100)')
plt.title('Customer Segments by Spending Score and Age')
plt.legend(*scatter.legend_elements(), title='Cluster', loc='upper right',
           bbox_to_anchor=(1.15, 1), fontsize='small')
plt.show()

## Step 9: Interpret the clusters (customer segments in the market)

In [ ]:
print("\nGender distribution by cluster:")
print(pd.crosstab(customer_data['Cluster'], customer_data['Genre']))
print("\nCluster statistics:")
print("\nMeans:")
display(customer_data.groupby('Cluster').mean(numeric_only=True))
print("\nStandard deviations:")
display(customer_data.groupby('Cluster').std(numeric_only=True))


| Cluster | Typical age | Gender | Income level | Spending Behaviour |
|---------|-------------|--------|--------------|--------------------|
| 0       | Young       | Both   | High         | Targeted premium   |
| 1       | Young       | Both   | Low–medium   | Average spenders   |
| 2       | Mature      | Both   | High         | Low spenders       |
| 3       | Mature      | Female | Low          | Frugal             |
| 4       | Middle‑age  | Both   | Medium       | Luxury shoppers    |

## Lab Submission Requirements

1. Provide the code to download the `'./data/mall_customers_with_clusters.csv'` file, which contains the original dataset with an additional column for the cluster labels assigned by the K-Means algorithm. This code should work in Google Colab.
2. Provide the code to train a Support Vector Machine (SVM) model using the `X` and `y` variables, where `X` is the feature matrix and `y` is the target variable (the cluster labels). The SVM model should be trained to predict the cluster labels based on the features.
3. Provide the code to demonstrate the prediction of cluster labels using the trained SVM model. This should include a sample input and the predicted cluster label for that input.
4. Submit the link to your notebook via the Google Form provided on Google Classroom. Ensure that you indicate your name and student ID in the notebook.